In [ ]:
'''
https://portal.icuregswe.org/utdata/sv/report/prod.belaggning

REMEMBER to add new,additional years to list years below 
'''

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

sns.set()

In [ ]:
all_data = pd.DataFrame()

In [ ]:
def read_occupacy(fname):
    excel_file = pd.ExcelFile(fname)
    
    sheet = excel_file.sheet_names
    
    df = excel_file.parse(sheet_names=sheet,skiprows=1)
    df.columns = ['Datum','Platser'] ### sometimes get english columns ###
    df['date'] = pd.to_datetime(df['Datum'])
    df.drop('Datum',inplace=True,axis=1)
    df.set_index('date',inplace=True)
    df.rename(columns={'Platser' : 'occupacy'},inplace=True)
    df = df.resample('D').mean()
    
    
    return (df)

In [ ]:
years = ['2016','2017','2018','2019','2020','2021','2022']

In [ ]:
for y in years:
    df = read_occupacy('iva_occupacy_' + y + '.xlsx')
    all_data = pd.concat([all_data,df])


In [ ]:
all_data

In [ ]:
all_data.index.dayofyear

In [ ]:
all_data.loc['2021'].index.isocalendar().week

In [ ]:
last_good_day = (all_data.index[-1] - pd.DateOffset(weeks=2))
last_good_day_nr = last_good_day.dayofyear

In [ ]:
plt.figure(figsize=(18,12))

for y in years:
    plt.plot(all_data.loc[y].index.dayofyear,all_data.loc[y],label=y)
    
plt.xlabel('day of year')
plt.ylabel('nr of ICU patients')
plt.axvline(last_good_day_nr,label='last stable day {}'.format(last_good_day.date()),color='red',ls='dashed')
plt.legend(loc='upper left')
plt.title('SWEDEN : Daily ICU occupacy ' + years[0] +' - YTD {}\n\nDataSource : Svenska Intensivvårdsregistret [Swedish ICU Registry]'.format(
last_good_day.date()))


In [ ]:
base_dates = all_data.loc[:'2019']
baseline = base_dates.groupby(base_dates.index.dayofyear).mean()
baseline

In [ ]:
fig,ax = plt.subplots(figsize=(18,12))

ax.plot(all_data.loc['2020'].index.dayofyear,all_data.loc['2020'],label='2020',color='red')
ax.plot(all_data.loc['2021'].index.dayofyear,all_data.loc['2021'],label='2021',color='green')
ax.plot(all_data.loc['2022'].index.dayofyear,all_data.loc['2022'],label='2022',color='blue')

ax.plot(baseline.index,baseline,label='baseline 2016-2019',ls='dashed',color='orange')
ax.axvline(last_good_day_nr,label='last stable day {}'.format(last_good_day.date()),color='k',ls='dashed')
ax.legend(loc='upper left')
ax.set_xlabel('day of year')
ax.set_ylabel('Nr of ICU patients')
plt.title('SWEDEN : Daily ICU occupacy 2020 - YTD {} cmp baseline 2016-2019\n\nDataSource : Svenska Intensivvårdsregistret [Swedish ICU Registry]'.format(
last_good_day.date()))
plt.savefig('iva_occupacy_vs_baseline.jpg',format='jpg')


In [ ]:
baseline

In [ ]:

ax = all_data.plot(figsize=(18,12))
ax.axvline(last_good_day,label='last stable day {}'.format(last_good_day.date()),color='k',ls='dashed')

non_leap_base = baseline.iloc[:-1]
leap_base = baseline

full_period_base = pd.concat([leap_base,non_leap_base,non_leap_base,non_leap_base,leap_base,non_leap_base,
                             non_leap_base])
full_period_base.index = pd.date_range('2016-01-01',periods=len(full_period_base))
full_period_base.plot(ax=ax,ls='dashed',color='orange')
ax.legend(['occupacy','stable data','baseline'])
full_period_base.to_pickle('iva_daily_baseline.pkl')

all_data.tail(40)

In [ ]:
all_data.info()

In [ ]:
arrow_properties = dict(
    facecolor="red", width=5,
    headwidth=10, shrink=1)

label_x = 18100
label_y = 400
#arrow_x = 18300
arrow_x = '2020-02-15'
arrow_x2 = '2020-09-15'
arrow_y = 350



cred = '\nDataSource : Svenska Intensivvårdsregistret'
title = 'SWEDEN ICU daily bed usage, rolling 7 day avg. 2016-2022 YTD, last stable date: {}'.format(last_good_day.date()) + cred
ax = all_data.rolling(7).mean().plot(figsize=(18,12),title=title)
ax.axvline(last_good_day,label='last stable day {}'.format(last_good_day.date()),color='k',ls='dashed')
full_period_base.rolling(7).mean().plot(ax=ax,ls='dashed',color='orange')


x_data = ax.get_xticks(minor=False)
y_data = ax.get_yticks()



print (x_data,y_data)

#print (ax.transData.transform((200,200)))

ax.annotate('Interesting Areas',xy=(arrow_x,arrow_y+20),
           xytext = (label_x,label_y + 40),color='red',
           arrowprops = arrow_properties)


ax.annotate('                          ',xy=(arrow_x2 ,arrow_y-20),
           xytext = (label_x ,label_y+40),color='red',
           arrowprops = arrow_properties)


'''
ax.annotate('Interesting \nArea',xy=(arrow_x,arrow_y+20),
           xytext = (label_x,label_y + 40),color='red',
           arrowprops = arrow_properties)

ax.annotate('Interesting \nArea',xy=(arrow_x + 225,arrow_y-20),
           xytext = (label_x + 225,label_y),color='red',
           arrowprops = arrow_properties)
'''

ax.legend(['ICU beds','last stable date','ICU beds baseline'])
ax.set_ylabel('beds in use')
ax.set_ylim([0,800])
plt.savefig('iva_usage_rolling.jpg',format='jpg')

In [ ]:
arrow_x = '2019-12-23'
arrow_x2 = '2020-03-27'
arrow_x3 = '2020-07-26'
arrow_x4 = '2020-10-23'

diff = (all_data.rolling(7).mean() - full_period_base.rolling(7).mean()).dropna()

title = 'SWEDEN Daily ICU usage : diff from baseline\nDataSource : Svenska Intensivvårdsregistret'
ax = diff.plot(figsize=(18,12),title=title)

ax.fill_between(x = diff.loc[arrow_x : arrow_x2].index,y1 = diff.loc[arrow_x : arrow_x2]['occupacy'],
               color='orange')
ax.fill_between(x = diff.loc[arrow_x3 : arrow_x4].index,y1 = diff.loc[arrow_x3 : arrow_x4]['occupacy'],
               color='orange')
ax.set_ylabel('difference')

plt.savefig('iva_diff_full.jpg',format='jpg')


diff = diff.loc['2019-10':'2021-12']
diff

In [ ]:



print (diff.loc[arrow_x : arrow_x2]['occupacy'])

title = 'SWEDEN Daily ICU usage : diff from baseline : {} - {} \nDataSource : Svenska Intensivvårdsregistret'.format(
diff.index[0].date(),diff.index[-1].date())

ax = diff.plot(figsize=(18,12),title=title)


ax.axvline(arrow_x,ls='dashed',color='red',label=arrow_x)
ax.axvline(arrow_x2,ls='dashed',color='red',label=arrow_x2)
ax.axvline(arrow_x3,ls='dashed',color='red',label=arrow_x3)
ax.axvline(arrow_x4,ls='dashed',color='red',label=arrow_x4)

ax.fill_between(x = diff.loc[arrow_x : arrow_x2].index,y1 = diff.loc[arrow_x : arrow_x2]['occupacy'],
               color='orange')
ax.fill_between(x = diff.loc[arrow_x3 : arrow_x4].index,y1 = diff.loc[arrow_x3 : arrow_x4]['occupacy'],
               color='orange')


ax.set_ylabel('diff actuals - baseline')
ax.legend(loc='upper right')

plt.savefig('iva_daily_diff.jpg')


In [ ]:
s = diff < 0
(~s).cumsum()[s].value_counts()



In [ ]:
### FHM ###
fhm_url = 'https://www.arcgis.com/sharing/rest/content/items/b5e7488e117749c19881cce45db13f7e/data'
fhm_file = pd.ExcelFile(fhm_url)
fhm_file.sheet_names


In [ ]:
fhm_icu = fhm_file.parse(sheet_name=fhm_file.sheet_names[2],index_col=0,parse_dates=True)
fhm_icu.rename(columns={'Antal_intensivvårdade' : 'ICU_Daily_New_Covid_cases'},inplace=True)
fhm_icu.index.names=['date']

In [ ]:
fhm_icu

In [ ]:
sir_fhm = all_data.join(fhm_icu).dropna()
sir_fhm

In [ ]:
sir_fhm['ICU_Daily_non_Covid_beds_in_use'] = sir_fhm['occupacy'] - sir_fhm['ICU_Daily_New_Covid_cases']
sir_fhm

In [ ]:
'''
Unable to draw any conclusions from the data since there's no info in the FHM data
on when a new case checks out
'''
ax = sir_fhm.plot(y=['ICU_Daily_New_Covid_cases','ICU_Daily_non_Covid_beds_in_use'],figsize=(18,12))


In [ ]:
'''
https://portal.icuregswe.org/siri/en/report/corona.covid-dagligen
'''
sir_covid_url = 'iva_covid_occupancy.xlsx'
sir_covid_file = pd.ExcelFile(sir_covid_url)
sir_covid_file.sheet_names

In [ ]:
sir_covid = sir_covid_file.parse(sheet_name=sir_covid_file.sheet_names[0],skiprows=1,index_col=0,parse_dates=True)
sir_covid = sir_covid['Covid-19 Totalt']
sir_covid

In [ ]:
sir_covid_non_covid = all_data.join(sir_covid).dropna()
sir_covid_non_covid

In [ ]:
sir_covid_non_covid['Non-Covid'] = sir_covid_non_covid['occupacy'] - sir_covid_non_covid['Covid-19 Totalt']
sir_covid_non_covid

In [ ]:
sir_covid_non_covid.rename(columns={'Covid-19 Totalt' : 'Covid','occupacy' : 'Total'},inplace=True)

In [ ]:
sir_covid_non_covid

In [ ]:
# daily average per week 
sir_covid_non_covid = sir_covid_non_covid.resample('W').mean()

In [ ]:
sir_covid_non_covid.index

In [ ]:
covid_period_all_icu_base = full_period_base.resample('W').mean()
covid_period_all_icu_base = covid_period_all_icu_base.loc[sir_covid_non_covid.index[0]:]
covid_period_all_icu_base

In [ ]:
print (covid_period_all_icu_base.info())
print()
sir_covid_non_covid.info()

In [ ]:
sir_covid_non_covid.index = pd.to_datetime(sir_covid_non_covid.index)
sir_covid_non_covid.index

In [ ]:
covid_period_all_icu_base.index

In [ ]:
fig,ax = plt.subplots(figsize=(18,12))
sir_covid_non_covid.index = sir_covid_non_covid.index
sir_covid_non_covid.plot(ax=ax,kind='bar',stacked=True,y=['Non-Covid','Covid'])


print (ax.get_xticks())
ax.axvline(ax.get_xticks()[-4],color='k',ls='dashed',label='last stable data')
ax.legend(loc='upper left')
ax.set_title('SWEDEN daily avg. ICU occupancy by week, Covid + Non-Covid, {} - {}, last stable date: {}\n\nDataSource : Svenska Intensivvårdsregistret (SIR,SIRI)'.format(
sir_covid_non_covid.index[0],sir_covid_non_covid.index[-1],sir_covid_non_covid.index[-4]))
ax.set_ylabel('beds occupied')
ax.set_xlabel('week of')


In [ ]:
combined_sir_and_base = sir_covid_non_covid.join(covid_period_all_icu_base)
combined_sir_and_base.rename(columns={'occupacy' : 'baseline'},inplace=True)
combined_sir_and_base

In [ ]:
combined_sir_and_base.info()

In [ ]:
'''
pandas bug: can't plot using datetime index a line and bar on same plot 
https://stackoverflow.com/questions/42455391/python-different-behaviour-of-datetimeindex-while-plotting-line-and-bar-plots
'''
combined_sir_and_base.index = combined_sir_and_base.index.astype(str)

In [ ]:
combined_sir_and_base.clip(lower=0)

In [ ]:
### reporting on covid specific beds is updated faster than the all cause beds. therefore ###
### the latest weeks can get negative ###

fig,ax = plt.subplots(figsize=(18,12))
combined_sir_and_base.clip(lower=0).plot(kind='bar',stacked=True,y=['Non-Covid','Covid'],ax=ax)
ax.plot(combined_sir_and_base.index,combined_sir_and_base['baseline'],ls='dashed',color='k',label='ICU total baseline')

ax.set_title('SWEDEN 2021 daily avg. ICU occupancy by week, Covid + Non-Covid, {} - {}, last stable date: {}\n\nDataSource : Svenska Intensivvårdsregistret (SIR,SIRI)\nData pull : {}'.format(sir_covid_non_covid.index[0].date(),sir_covid_non_covid.index[-2].date(),sir_covid_non_covid.index[-4].date(),
pd.to_datetime('today').date()))

print (ax.get_xticks())

ax.axvline(ax.get_xticks()[-4],color='red',ls='dashed',label='last stable data')


#ax.legend(['Non-Covid','Covid','ICU-baseline','last stable date'][::-1])
ax.legend(loc='upper center')
ax.set_ylabel('beds occupied')
ax.set_xlabel('week ending')


for label in ax.xaxis.get_ticklabels()[1::2]:
    label.set_visible(False)
   


plt.savefig('iva_covid_non_covid.jpg')

In [ ]:
from datetime import date

variants = pd.read_pickle('variants.pkl')
variants.index = pd.DatetimeIndex(pd.to_datetime(variants.index.str.strip() + '-01',format='%Y-%W-%w%f').date)
variants.index

In [ ]:
combined_sir_and_base.index = pd.DatetimeIndex(combined_sir_and_base.index)

combined_sir_base_variants = (combined_sir_and_base.join(variants))
combined_sir_base_variants.drop('year',axis=1,inplace=True)
combined_sir_base_variants.dropna(inplace=True)
combined_sir_base_variants

In [ ]:
arrow_properties = dict(
    facecolor="orange", width=5,
    headwidth=10, shrink=1)


fig,axes = plt.subplots(2,figsize=(18,12),sharex=True)

xticks = combined_sir_base_variants.index.date

combined_sir_base_variants.plot(y=['Non-Covid','Covid'],kind='bar',stacked=True,ax=axes[1])
combined_sir_base_variants.plot(y='baseline',ax=axes[1],kind='bar',facecolor='none',edgecolor='k')
combined_sir_base_variants.plot(y=['alpha','beta','gamma','delta','omicron'],kind='bar',stacked=True,ax=axes[0])

axes[1].set_title('SWEDEN 2021 --> daily avg. ICU occupancy by week, Covid + Non-Covid, {} - {}\nDataSource : Svenska Intensivvårdsregistret (SIR,SIRI)'.format(
    combined_sir_base_variants.index[0].date(),combined_sir_base_variants.index[-1].date()))

axes[1].set_ylabel('number of ICU beds in use')

_= axes[1].set_xticklabels(xticks)

_= axes[0].set_title('SWEDEN 2021 --> Covid Variants of Concern {} - {}\nDataSource : Folkhälsomyndigheten'.format(
combined_sir_base_variants.index[0].date(),combined_sir_base_variants.index[-1].date()))

axes[0].set_ylabel('nr of sequenced virus samples')
print (axes[0].get_xticks())


axes[0].annotate('DELTA Arrives',xy=(17,2100),
           xytext = (16,4000),color='orange',
           arrowprops = arrow_properties)

axes[1].annotate('DELTA Arrives',xy=(17,450),
           xytext = (17,600),color='orange',
           arrowprops = arrow_properties)

axes[0].annotate('OMICRON Arrives',xy=(44,3100),
           xytext = (44,4500),color='orange',
           arrowprops = arrow_properties)

axes[1].annotate('OMICRON Arrives',xy=(44,350),
           xytext = (44,500),color='orange',
           arrowprops = arrow_properties)


plt.savefig('scb_excess_icus_and_variants.jpg',format='jpg')